In [292]:
#model = load_model('static/data/nnn_model_1.h5')


s


    #return predict_input




In [293]:
lat= 41.14339925293028
lng=  -112.029677791514445
the_date = '2023-04-01'

#41.118140	-112.031425	62.482979	57.820677	3	1	2

In [294]:
pm25_predict(lat,lng,the_date)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


array([[ 1.53839792, -1.73281768,  0.10237968,  0.08796155,  1.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ]])

In [175]:
xdf = pd.read_csv('../static/data/centroids_data.csv')

In [29]:
pd.read_csv('../static/data/centroids_data.csv').to_json('../static/data/centroids_data.json', orient='records')


In [39]:
round(lng,5)

-111.89747

In [46]:
url = '../static/data/centroid_prep.csv'
temp_centroid = pd.read_csv(url)
temp_centroid['latitude'] = temp_centroid['latitude'].round(5)
temp_centroid['longitude'] = temp_centroid['longitude'].round(5)

In [45]:
temp_centroid

,Unnamed: 0,OBJECTID,GEOID,Tract,Low,Lowmod,Lowmod_pct,geometry,longitude,latitude,County_encoded,category_encoded,category_blue,category_green,category_orange,category_red,county_Davis County,county_Salt Lake County,county_Weber County
0,0,199060,490111251021,125102,120,340,0.1833,POINT (-111.88080951152304 41.08032456289652),-111.880810,41.080325,0,0,True,False,False,False,True,False,False
1,1,199061,490111251022,125102,245,490,0.1892,POINT (-111.88003663259194 41.04248386241919),-111.880037,41.042484,0,0,True,False,False,False,True,False,False
2,2,199062,490111251031,125103,370,850,0.3131,POINT (-111.9563088264819 41.13919426676789),-111.956309,41.139194,0,1,False,True,False,False,True,False,False
3,3,199063,490111251032,125103,430,1000,0.4264,POINT (-111.94339340147886 41.11547671683034),-111.943393,41.115477,0,1,False,True,False,False,True,False,False
4,4,199064,490111251041,125104,165,475,0.1967,POINT (-111.91575298025643 41.12877405887183),-111.915753,41.128774,0,0,True,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,907,200607,490572112012,211201,70,170,0.1560,POINT (-111.92049976396636 41.15200853107149),-111.920500,41.152009,2,0,True,False,False,False,False,False,True
908,908,200608,490572112013,211201,365,975,0.4194,POINT (-111.93262761353571 41.156133092177804),-111.932628,41.156133,2,1,False,True,False,False,False,False,True
909,909,200609,490572112021,211202,115,270,0.3354,POINT (-111.96410356104677 41.16503488421328),-111.964104,41.165035,2,1,False,True,False,False,False,False,True
910,910,200610,490572112022,211202,620,1710,0.3963,POINT (-111.95158835880763 41.15407118264731),-111.951588,41.154071,2,1,False,True,False,False,False,False,True
